In [ ]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node
from pychunkedgraph.graph import cache

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MergeOperation

cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [ ]:
data = {
    "sources": [["94803466563722624",916778,851942.625,783023.25]], # 354
    "sinks": [["94803535283198313",916697.1875,852031.75,783023.25]], # 832 = 1186
}
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict

In [ ]:
edges = [
    [data_dict["sources"]["id"][0], data_dict["sinks"]["id"][0]],
]
roots_old = cg.get_roots(edges[0])
print(edges[0], roots_old)
print(cg.get_cross_chunk_edges_layer(edges))

# dfs_print_node(cg, roots_old[0], limit=2, sep="    ", count_only=True)
# dfs_print_node(cg, roots_old[1], limit=2, sep="    ", count_only=True)

In [ ]:
cache.clear()
cg.cache = cache.CacheService(cg)

op = MergeOperation(
    cg,
    user_id="test",
    added_edges=edges,
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],
)

with TimeIt("add_edges"):
    new_ids = op._apply(operation_id="",timestamp=None)

In [ ]:
print()
roots = new_ids[12]
for i in range(2, 13):
    print(i, new_ids[i])

dfs_print_node(cg, roots[0], limit=2, sep="    ", count_only=True)